In [1]:
import pandas as pd

In [2]:
tickers = pd.read_csv('ticker_list.csv')
tickernames = tickers['name']
biotech_tickers = tickernames.to_list()

In [3]:
import random
import string

def random_string(length):
    x = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(length))
    return x

random_string(13)

'nuf68vv282mkh'

In [5]:
import requests

In [6]:
for ticker in biotech_tickers:
    query = {'s': random_string(13), 't': ticker}
    res = requests.get('https://www.macrotrends.net/assets/php/stock_data_download.php', params=query)
    decoded = res.content.decode('utf-8')
    with open(f'tickers/{ticker}.csv', 'w') as f:
        f.write(decoded)
        f.close()

## Getting data for the same time period

At this point, we have CSV files for each stock saved as ticker.csv in the `tickers` folder in this directory.

### Remove the download message

The real data starts at the CSV column headings, with `date`, so we can remove everything before that

In [7]:
with open('tickers/ABCM.csv', 'r+') as f:
    content = f.read()
    date_index = content.index('date')
    cropped = content[date_index:]
    f.seek(0)
    f.write(cropped)
    f.close()

In [8]:
for ticker in biotech_tickers:
    file_path = f'tickers/{ticker}.csv'
    with open(file_path, 'r+') as f:
        content = f.read()
        date_index = content.index('date')
        cropped = content[date_index:]
        f.seek(0)
        f.write(cropped)
        f.close()

## Reduce data to just the opening prices

In [9]:
ticker = 'ABCL'
file_path = f'tickers/{ticker}.csv'
df = pd.read_csv(file_path)
df = df.drop(['high', 'low', 'close', 'volume'], axis=1)
df.to_csv(file_path)

In [10]:
for ticker in biotech_tickers:
    file_path = f'tickers/{ticker}.csv'
    df = pd.read_csv(file_path)
    try:
        df = df.drop(['high', 'low', 'close', 'volume'], axis=1)
        df.to_csv(file_path)
    except Exception as e:
        print(f'Error for {ticker}: {e}')

Error for ABCL: "['high', 'low', 'close', 'volume'] not found in axis"


## Reducing data to a timeframe

In [11]:
for year in range(2013, 2023, 1):
    count = 0
    for ticker in biotech_tickers:
        file_path = f'tickers/{ticker}.csv'
        df = pd.read_csv(file_path)
        earliest_date = df['date'][0]
        earliest_date_year = int(earliest_date[:4])
        if earliest_date_year <= year:
            count += 1

    print(f'{count} stocks have data starting in a year before {year}')

34 stocks have data starting in a year before 2013
37 stocks have data starting in a year before 2014
50 stocks have data starting in a year before 2015
55 stocks have data starting in a year before 2016
65 stocks have data starting in a year before 2017
78 stocks have data starting in a year before 2018
97 stocks have data starting in a year before 2019
126 stocks have data starting in a year before 2020
158 stocks have data starting in a year before 2021
163 stocks have data starting in a year before 2022


This year, we're going with 2015 onwards

In [12]:
year = 2015

In [13]:
tickers_2015 = []

for ticker in biotech_tickers:
    df = pd.read_csv(f'tickers/{ticker}.csv')
    earliest_date = df['date'][0]
    earliest_date_year = int(earliest_date[:4])
    if earliest_date_year <= year:
        df.to_csv(f'tickers_2015/{ticker}.csv')
        tickers_2015.append(ticker)

Now, we need to remove values before 1 Jan 2015

In [14]:
def date_str_valid(date_str):
    try:
        year = int(str(date_str)[:4])
    except:
        return False
    if year >= 2015:
        return True
    return False    

In [15]:
df = df[df.apply(lambda x: date_str_valid(x['date']), axis = 1)]
df

,Unnamed: 0,date,open
0,0,2020-01-29,6.6300
1,1,2020-01-30,9.0000
2,2,2020-01-31,6.9800
3,3,2020-02-03,7.1500
4,4,2020-02-04,7.1800
...,...,...,...
669,669,2022-08-29,11.0100
670,670,2022-08-30,11.6000
671,671,2022-08-31,11.0400
672,672,2022-09-01,11.3147


In [16]:
for ticker in tickers_2015:
    original_file_path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(original_file_path)
    df = df.dropna()
    df = df[df.apply(lambda x: date_str_valid(x['date']), axis = 1)]
    df.to_csv(original_file_path)

Now we need to remove data from 2020 or later

In [17]:
def date_str_before_2020(date_str):
    try:
        year = int(str(date_str)[:4])
    except:
        return False
    if year < 2020:
        return True
    return False    

In [18]:
for ticker in tickers_2015:
    original_file_path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(original_file_path)
    df = df.dropna()
    df = df[df.apply(lambda x: date_str_before_2020(x['date']), axis = 1)]
    df.to_csv(original_file_path)

We now have a bunch of junk which we should remove from each of the files

In [19]:
for ticker in tickers_2015:
    file_path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(file_path)
    df = df[['date', 'open']]
    df.to_csv(file_path)

## Preparing Data

Now we need to merge these all into one dataframe / CSV file. This should have the date as the column and opening price for that stock as the row

First, we need to change the files so that each have them have the date as the column and opening price as the value

In [20]:
for ticker in tickers_2015:
    path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(path)
    df = df[['date', 'open']]
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df.drop(['date'])
    df.insert(loc=0, column='', value=ticker)
    df.to_csv(path, index=False)

Now, we need to merge all the dataframes into one mega DF

In [21]:
dfs = []

for ticker in tickers_2015:
    path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(path)
    dfs.append(df)

big_boi = pd.concat(dfs, axis=0)
big_boi

,Unnamed: 0,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-14,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
0,NVO,36.1597,36.2022,35.9048,35.8878,36.2022,36.6102,37.0691,37.4345,37.5110,...,54.9628,54.4045,54.1207,54.8965,54.8965,54.6221,54.8397,55.4642,55.2750,54.5748
0,VRTX,119.9700,122.2700,121.3100,118.8200,122.0000,123.7500,123.0100,120.4000,118.4200,...,219.7200,217.8300,220.1400,219.6200,220.9500,220.4600,220.6000,220.5900,220.3800,218.6000
0,REGN,413.9000,414.2600,414.3900,402.7900,411.5700,409.7800,414.0200,417.7300,403.1200,...,375.7300,370.3000,371.9500,374.9400,377.2000,374.7500,376.7600,375.2400,373.7400,373.6400
0,RPRX,10.0200,9.5700,9.6100,9.0900,8.9700,8.9000,9.3500,9.1400,8.6600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,SGEN,32.3100,32.3800,33.0400,31.8400,32.4900,31.7400,31.8800,32.2000,31.0200,...,115.2000,113.9600,118.5000,115.0000,118.3200,115.2100,115.1700,113.7300,113.6900,113.9000
0,ALNY,97.9700,96.4400,95.7700,94.8500,97.3500,102.1400,103.2200,105.5700,102.4100,...,123.7000,117.2700,116.1000,117.8800,115.7400,117.0800,118.0000,118.3000,116.8300,114.0000
0,BMRN,91.2400,90.8900,93.1100,91.7200,95.6600,94.4400,95.3100,97.2900,95.4400,...,83.3600,83.1300,84.6100,85.4500,85.3000,86.1100,86.4300,86.1600,85.5500,84.1500
0,INCY,73.6500,73.8200,73.2300,72.4800,75.0000,73.0100,73.0700,73.9200,75.2800,...,93.4300,90.8400,90.9000,92.0100,90.6800,90.4700,91.2000,89.1700,88.2700,87.7000
0,TECH,87.3801,85.4824,85.2475,84.8624,85.9991,85.7830,86.4783,86.7225,86.1964,...,217.1782,215.2583,213.8926,216.2084,219.9591,220.5529,219.0190,216.4459,215.8224,215.1791
0,UTHR,129.5000,126.8400,127.1100,125.5300,126.8700,128.8800,129.7800,133.9200,133.9200,...,91.2600,89.5800,89.8200,89.4900,90.7600,90.8900,91.2700,89.6400,88.6500,86.9100


In [22]:
big_boi.reset_index()
big_boi

,Unnamed: 0,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-14,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
0,NVO,36.1597,36.2022,35.9048,35.8878,36.2022,36.6102,37.0691,37.4345,37.5110,...,54.9628,54.4045,54.1207,54.8965,54.8965,54.6221,54.8397,55.4642,55.2750,54.5748
0,VRTX,119.9700,122.2700,121.3100,118.8200,122.0000,123.7500,123.0100,120.4000,118.4200,...,219.7200,217.8300,220.1400,219.6200,220.9500,220.4600,220.6000,220.5900,220.3800,218.6000
0,REGN,413.9000,414.2600,414.3900,402.7900,411.5700,409.7800,414.0200,417.7300,403.1200,...,375.7300,370.3000,371.9500,374.9400,377.2000,374.7500,376.7600,375.2400,373.7400,373.6400
0,RPRX,10.0200,9.5700,9.6100,9.0900,8.9700,8.9000,9.3500,9.1400,8.6600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,SGEN,32.3100,32.3800,33.0400,31.8400,32.4900,31.7400,31.8800,32.2000,31.0200,...,115.2000,113.9600,118.5000,115.0000,118.3200,115.2100,115.1700,113.7300,113.6900,113.9000
0,ALNY,97.9700,96.4400,95.7700,94.8500,97.3500,102.1400,103.2200,105.5700,102.4100,...,123.7000,117.2700,116.1000,117.8800,115.7400,117.0800,118.0000,118.3000,116.8300,114.0000
0,BMRN,91.2400,90.8900,93.1100,91.7200,95.6600,94.4400,95.3100,97.2900,95.4400,...,83.3600,83.1300,84.6100,85.4500,85.3000,86.1100,86.4300,86.1600,85.5500,84.1500
0,INCY,73.6500,73.8200,73.2300,72.4800,75.0000,73.0100,73.0700,73.9200,75.2800,...,93.4300,90.8400,90.9000,92.0100,90.6800,90.4700,91.2000,89.1700,88.2700,87.7000
0,TECH,87.3801,85.4824,85.2475,84.8624,85.9991,85.7830,86.4783,86.7225,86.1964,...,217.1782,215.2583,213.8926,216.2084,219.9591,220.5529,219.0190,216.4459,215.8224,215.1791
0,UTHR,129.5000,126.8400,127.1100,125.5300,126.8700,128.8800,129.7800,133.9200,133.9200,...,91.2600,89.5800,89.8200,89.4900,90.7600,90.8900,91.2700,89.6400,88.6500,86.9100


Seems like some rows are missing in `big_boi`

In [23]:
import numpy as np
columns_all = np.array([column for column in [columns for columns in [df.columns for df in dfs]]])
first_dates = [i[1] for i in columns_all]
(first_dates.sort())
first_dates[-1]

/var/folders/8m/xxtd2bpx07b2133k5m87vfq80000gn/T/ipykernel_21630/1148072085.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  columns_all = np.array([column for column in [columns for columns in [df.columns for df in dfs]]])


'2015-12-22'

The latest first date is  '2015-12-22'. This means we will have to reduce all of our data so that it starts on the 22nd of December 2015.

In [24]:
dates_to_remove = []
for i in range(1, 12, 1):
    month_code = ''
    if i <= 9:
        month_code = f'0{i}'
    else:
        month_code = str(i)
    for k in range(1, 32, 1):
        if k <= 9:
            date = f'2015-{month_code}-0{k}'
        else:
            date = f'2015-{month_code}-{k}'
        dates_to_remove.append(date)

for j in range(1, 22, 1):
    if j <= 9:
        date = f'2015-12-0{j}'
    else:
        date = f'2015-12-{j}'
    dates_to_remove.append(date)

In [25]:
'2015-10-06' in dates_to_remove

True

In [26]:
dfs = []

for ticker in tickers_2015:
    path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(path)
    df = df.drop(dates_to_remove, axis=1, errors='ignore')
    dfs.append(df)

big_boi = pd.concat(dfs, axis=0)
big_boi

,Unnamed: 0,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-04,2016-01-05,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
0,NVO,49.0179,49.5701,49.7771,50.2947,50.4069,50.6312,50.2947,49.8548,50.0100,...,54.9628,54.4045,54.1207,54.8965,54.8965,54.6221,54.8397,55.4642,55.2750,54.5748
0,VRTX,121.0900,123.9200,124.3800,123.9900,124.9700,127.6800,125.5500,123.0300,123.8900,...,219.7200,217.8300,220.1400,219.6200,220.9500,220.4600,220.6000,220.5900,220.3800,218.6000
0,REGN,535.5900,531.0600,535.5600,536.4500,542.2800,550.5400,542.6200,531.7600,519.3800,...,375.7300,370.3000,371.9500,374.9400,377.2000,374.7500,376.7600,375.2400,373.7400,373.6400
0,RPRX,1.2500,1.2700,1.3400,1.2600,1.2400,1.1300,1.1600,1.1900,1.2900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,SGEN,42.3100,42.7300,43.7100,43.7000,44.5100,45.3000,44.8200,43.8800,42.0700,...,115.2000,113.9600,118.5000,115.0000,118.3200,115.2100,115.1700,113.7300,113.6900,113.9000
0,ALNY,88.2700,87.2400,91.2900,90.1600,92.4900,93.4800,94.1800,92.1800,93.4900,...,123.7000,117.2700,116.1000,117.8800,115.7400,117.0800,118.0000,118.3000,116.8300,114.0000
0,BMRN,104.3600,104.4000,105.8300,105.9100,106.0200,106.3000,105.4400,102.8000,105.0200,...,83.3600,83.1300,84.6100,85.4500,85.3000,86.1100,86.4300,86.1600,85.5500,84.1500
0,INCY,113.7800,109.2100,110.2600,108.9800,109.9800,110.2700,108.2400,106.5000,102.1400,...,93.4300,90.8400,90.9000,92.0100,90.6800,90.4700,91.2000,89.1700,88.2700,87.7000
0,TECH,83.8159,85.2154,85.3011,86.6815,87.7763,87.4241,86.4816,84.7965,84.0635,...,217.1782,215.2583,213.8926,216.2084,219.9591,220.5529,219.0190,216.4459,215.8224,215.1791
0,UTHR,158.4200,159.2800,159.1500,159.9900,161.8700,159.2600,157.7800,154.2300,156.3800,...,91.2600,89.5800,89.8200,89.4900,90.7600,90.8900,91.2700,89.6400,88.6500,86.9100


Now for some cleaning

In [27]:
big_boi = big_boi.reset_index(drop=True)

In [28]:
big_boi.rename(columns={'Unnamed: 0': 'ticker'}, inplace=True)
big_boi.head()

,ticker,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-04,2016-01-05,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
0,NVO,49.0179,49.5701,49.7771,50.2947,50.4069,50.6312,50.2947,49.8548,50.01,...,54.9628,54.4045,54.1207,54.8965,54.8965,54.6221,54.8397,55.4642,55.275,54.5748
1,VRTX,121.0900,123.9200,124.3800,123.9900,124.9700,127.6800,125.5500,123.0300,123.89,...,219.7200,217.8300,220.1400,219.6200,220.9500,220.4600,220.6000,220.5900,220.380,218.6000
2,REGN,535.5900,531.0600,535.5600,536.4500,542.2800,550.5400,542.6200,531.7600,519.38,...,375.7300,370.3000,371.9500,374.9400,377.2000,374.7500,376.7600,375.2400,373.740,373.6400
3,RPRX,1.2500,1.2700,1.3400,1.2600,1.2400,1.1300,1.1600,1.1900,1.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SGEN,42.3100,42.7300,43.7100,43.7000,44.5100,45.3000,44.8200,43.8800,42.07,...,115.2000,113.9600,118.5000,115.0000,118.3200,115.2100,115.1700,113.7300,113.690,113.9000


Seems like some values are still missing... because the stock market be like that. Let's nuke them

In [29]:
big_boi = big_boi.dropna()
big_boi

,ticker,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-04,2016-01-05,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
0,NVO,49.0179,49.5701,49.7771,50.2947,50.4069,50.6312,50.2947,49.8548,50.0100,...,54.9628,54.4045,54.1207,54.8965,54.8965,54.6221,54.8397,55.4642,55.2750,54.5748
1,VRTX,121.0900,123.9200,124.3800,123.9900,124.9700,127.6800,125.5500,123.0300,123.8900,...,219.7200,217.8300,220.1400,219.6200,220.9500,220.4600,220.6000,220.5900,220.3800,218.6000
2,REGN,535.5900,531.0600,535.5600,536.4500,542.2800,550.5400,542.6200,531.7600,519.3800,...,375.7300,370.3000,371.9500,374.9400,377.2000,374.7500,376.7600,375.2400,373.7400,373.6400
4,SGEN,42.3100,42.7300,43.7100,43.7000,44.5100,45.3000,44.8200,43.8800,42.0700,...,115.2000,113.9600,118.5000,115.0000,118.3200,115.2100,115.1700,113.7300,113.6900,113.9000
5,ALNY,88.2700,87.2400,91.2900,90.1600,92.4900,93.4800,94.1800,92.1800,93.4900,...,123.7000,117.2700,116.1000,117.8800,115.7400,117.0800,118.0000,118.3000,116.8300,114.0000
6,BMRN,104.3600,104.4000,105.8300,105.9100,106.0200,106.3000,105.4400,102.8000,105.0200,...,83.3600,83.1300,84.6100,85.4500,85.3000,86.1100,86.4300,86.1600,85.5500,84.1500
7,INCY,113.7800,109.2100,110.2600,108.9800,109.9800,110.2700,108.2400,106.5000,102.1400,...,93.4300,90.8400,90.9000,92.0100,90.6800,90.4700,91.2000,89.1700,88.2700,87.7000
8,TECH,83.8159,85.2154,85.3011,86.6815,87.7763,87.4241,86.4816,84.7965,84.0635,...,217.1782,215.2583,213.8926,216.2084,219.9591,220.5529,219.0190,216.4459,215.8224,215.1791
9,UTHR,158.4200,159.2800,159.1500,159.9900,161.8700,159.2600,157.7800,154.2300,156.3800,...,91.2600,89.5800,89.8200,89.4900,90.7600,90.8900,91.2700,89.6400,88.6500,86.9100
10,JAZZ,139.9900,144.1500,144.2400,141.7800,142.6500,142.5600,142.5500,137.8300,138.3500,...,149.2200,149.2300,149.6200,153.0000,153.9500,153.5700,153.6200,152.4500,151.3200,148.0900


In [30]:
big_boi.to_csv('opening_prices_biotech_original.csv')

Now all that's left is we need to remove the explicit date headings and instead label the dates as relative to the first date. 

In [31]:
num_days = len(big_boi.iloc[0]) - 1
column_names = ['ticker']
for i in range(num_days):
    day_id = f'day_{i}'
    column_names.append(day_id)
column_names

['ticker',
 'day_0',
 'day_1',
 'day_2',
 'day_3',
 'day_4',
 'day_5',
 'day_6',
 'day_7',
 'day_8',
 'day_9',
 'day_10',
 'day_11',
 'day_12',
 'day_13',
 'day_14',
 'day_15',
 'day_16',
 'day_17',
 'day_18',
 'day_19',
 'day_20',
 'day_21',
 'day_22',
 'day_23',
 'day_24',
 'day_25',
 'day_26',
 'day_27',
 'day_28',
 'day_29',
 'day_30',
 'day_31',
 'day_32',
 'day_33',
 'day_34',
 'day_35',
 'day_36',
 'day_37',
 'day_38',
 'day_39',
 'day_40',
 'day_41',
 'day_42',
 'day_43',
 'day_44',
 'day_45',
 'day_46',
 'day_47',
 'day_48',
 'day_49',
 'day_50',
 'day_51',
 'day_52',
 'day_53',
 'day_54',
 'day_55',
 'day_56',
 'day_57',
 'day_58',
 'day_59',
 'day_60',
 'day_61',
 'day_62',
 'day_63',
 'day_64',
 'day_65',
 'day_66',
 'day_67',
 'day_68',
 'day_69',
 'day_70',
 'day_71',
 'day_72',
 'day_73',
 'day_74',
 'day_75',
 'day_76',
 'day_77',
 'day_78',
 'day_79',
 'day_80',
 'day_81',
 'day_82',
 'day_83',
 'day_84',
 'day_85',
 'day_86',
 'day_87',
 'day_88',
 'day_89',
 'day_90'

In [32]:
big_boi.columns = column_names
big_boi

,ticker,day_0,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,...,day_1003,day_1004,day_1005,day_1006,day_1007,day_1008,day_1009,day_1010,day_1011,day_1012
0,NVO,49.0179,49.5701,49.7771,50.2947,50.4069,50.6312,50.2947,49.8548,50.0100,...,54.9628,54.4045,54.1207,54.8965,54.8965,54.6221,54.8397,55.4642,55.2750,54.5748
1,VRTX,121.0900,123.9200,124.3800,123.9900,124.9700,127.6800,125.5500,123.0300,123.8900,...,219.7200,217.8300,220.1400,219.6200,220.9500,220.4600,220.6000,220.5900,220.3800,218.6000
2,REGN,535.5900,531.0600,535.5600,536.4500,542.2800,550.5400,542.6200,531.7600,519.3800,...,375.7300,370.3000,371.9500,374.9400,377.2000,374.7500,376.7600,375.2400,373.7400,373.6400
4,SGEN,42.3100,42.7300,43.7100,43.7000,44.5100,45.3000,44.8200,43.8800,42.0700,...,115.2000,113.9600,118.5000,115.0000,118.3200,115.2100,115.1700,113.7300,113.6900,113.9000
5,ALNY,88.2700,87.2400,91.2900,90.1600,92.4900,93.4800,94.1800,92.1800,93.4900,...,123.7000,117.2700,116.1000,117.8800,115.7400,117.0800,118.0000,118.3000,116.8300,114.0000
6,BMRN,104.3600,104.4000,105.8300,105.9100,106.0200,106.3000,105.4400,102.8000,105.0200,...,83.3600,83.1300,84.6100,85.4500,85.3000,86.1100,86.4300,86.1600,85.5500,84.1500
7,INCY,113.7800,109.2100,110.2600,108.9800,109.9800,110.2700,108.2400,106.5000,102.1400,...,93.4300,90.8400,90.9000,92.0100,90.6800,90.4700,91.2000,89.1700,88.2700,87.7000
8,TECH,83.8159,85.2154,85.3011,86.6815,87.7763,87.4241,86.4816,84.7965,84.0635,...,217.1782,215.2583,213.8926,216.2084,219.9591,220.5529,219.0190,216.4459,215.8224,215.1791
9,UTHR,158.4200,159.2800,159.1500,159.9900,161.8700,159.2600,157.7800,154.2300,156.3800,...,91.2600,89.5800,89.8200,89.4900,90.7600,90.8900,91.2700,89.6400,88.6500,86.9100
10,JAZZ,139.9900,144.1500,144.2400,141.7800,142.6500,142.5600,142.5500,137.8300,138.3500,...,149.2200,149.2300,149.6200,153.0000,153.9500,153.5700,153.6200,152.4500,151.3200,148.0900


In [33]:
big_boi.to_csv('biotech_50.csv')

# Splitting data
We  need to save about 60 days worth of data for ourselves

In [49]:
last_day_id = 1012
col_names = ['ticker']
for i in range(last_day_id-61, last_day_id, 1):
    col_name = f'day_{i}'
    col_names.append(col_name)

In [51]:
testing_set = big_boi[col_names]
testing_set.to_csv('testing_set.csv')

In [52]:
last_day_id = 1012 - 61
col_names = ['ticker']
for i in range(0, last_day_id, 1):
    col_name = f'day_{i}'
    col_names.append(col_name)

In [55]:
training_set = big_boi[col_names]
training_set.to_csv('opening_prices_biotech.csv')